# Third step in pipeline
# Get topics of predicted segments

In [2]:
import numpy as np

In [51]:
# Load boundaries and probability desiti vectors of corresponding transcript
hdbscan_model_number = 50
transcript_ref = '13NDTKL5ZGs8cb8dojW3bz'

boundaries = np.load(f'../Thesis/evaluation/boundaries_msize{hdbscan_model_number}_{transcript_ref}.npy')
proba_vectors = np.load(f'../Thesis/annotated_probabilities_msize{hdbscan_model_number}/proba_dens_vec_msiz{hdbscan_model_number}{transcript_ref}.npy')
print(len(boundaries))
print(proba_vectors.shape)

6
(224, 555)


In [52]:
# For each segment, add probability desity vectors and find max index which matches with the topic 
certainty_ls = []

for index, bound in enumerate(boundaries):
    if index == len(boundaries)-1:
        segment_before = proba_vectors[boundaries[index-1]:bound+1,:]
        segment_after = proba_vectors[bound+1:,:]
        segment_before_added = np.add.reduce(segment_before)
        segment_after_added = np.add.reduce(segment_after)
        max_topic_before = np.argmax(segment_before_added)  # index of largest element
        max_topic_after = np.argmax(segment_after_added)  # index of largest element
        norm_value_before = np.max(segment_before_added)/len(segment_before)  # value of largest element
        norm_value_after = np.max(segment_after_added)/len(segment_after)  # value of largest element
        certainty_ls.append([norm_value_before, len(segment_before)])
        certainty_ls.append([norm_value_after, len(segment_after)])
        print(f'Segment before position {bound} has topic: {max_topic_before} and normalised value: {norm_value_before:.3f}')
        print(f'Segment after position {bound} has topic: {max_topic_after} and normalised value: {norm_value_after:.3f}')
    elif index == 0:
        segment = proba_vectors[:bound+1,:]
        segment_added = np.add.reduce(segment)
        max_topic = np.argmax(segment_added)  # index of largest element
        norm_value = np.max(segment_added)/len(segment)  # value of largest element
        certainty_ls.append([norm_value, len(segment)])
        print(f'Segment before position {bound} has topic: {max_topic} and normalised value: {norm_value:.3f}')
    else:
        segment = proba_vectors[boundaries[index-1]:bound+1,:]
        segment_added = np.add.reduce(segment)
        max_topic = np.argmax(segment_added)  # index of largest element
        norm_value = np.max(segment_added)/len(segment)  # value of largest element
        certainty_ls.append([norm_value, len(segment)])
        print(f'Segment before position {bound} has topic: {max_topic} and normalised value: {norm_value:.3f}')

# Calculate weighted average certainty
certainty_arr = np.array(certainty_ls)
weights = certainty_arr[:,1]/sum(certainty_arr[:,1])
weighted_average = sum(certainty_arr[:,0] * weights)
print(f'Weighted average certainty: {weighted_average:.1%}')

Segment before position 64 has topic: 81 and normalised value: 0.264
Segment before position 84 has topic: 419 and normalised value: 0.116
Segment before position 130 has topic: 81 and normalised value: 0.290
Segment before position 150 has topic: 81 and normalised value: 0.113
Segment before position 168 has topic: 81 and normalised value: 0.313
Segment before position 189 has topic: 320 and normalised value: 0.090
Segment after position 189 has topic: 81 and normalised value: 0.179
Weighted average certainty: 21.7%
